Importamos las librerias

In [2]:
import numpy as np
import pandas as pd

Cargamos nuestra data

In [3]:
data_games=pd.read_csv('games.csv')
data_review=pd.read_csv('reviews.csv')
data_items=pd.read_csv('items.csv')

Vamos a crear a una data auxiliar que contengan las columnas item_id y year del dataFrame data_review

In [200]:
data_auxiliar=data_review[['item_id','year']]

In [201]:
data_auxiliar.tail(2)

,item_id,year
49157,730,2015
49158,369200,2015


Eliminamos las filas que tenga item_id y year duplicados

In [167]:
data_auxiliar=data_auxiliar.drop_duplicates(subset=['item_id','year'])

Agrupamos los item_id y nos quedamos con los del primer año

In [168]:
data_auxiliar=data_auxiliar.groupby('item_id')['year'].min().reset_index()

In [169]:
data_auxiliar.head(2)

,item_id,year
0,10,2011
1,20,2012


Vamos a unir el `data_auxiliar` con `data_items` por medio de la columna `item_id`. Esto con el fin de poder trabajar las funciones pedidas.

## **Importante**

Para poder realizar la unión entre los dos DataFrames, es necesario tomar una muestra del DataFrame `data_items`. Debido a la cantidad de datos que posee este último, es complejo realizar un merge con el DataFrame normal.

In [170]:
data_items.tail(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,genres_games
4188783,76561198107283457,33,76561198107283457,http://steamcommunity.com/profiles/76561198107...,354280,ChaosTower,164,0,Action
4188784,inven,3603,76561198119803063,http://steamcommunity.com/id/inven,433920,Aveyond 4: Shadow Of The Mist,0,0,Adventure


In [171]:
muestra_items=data_items.sample(n=500000, random_state=42) # tomamos una muestra de 500000 datos

In [172]:
muestra_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 4104704 to 3208817
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           500000 non-null  object
 1   items_count       500000 non-null  int64 
 2   steam_id          500000 non-null  int64 
 3   user_url          500000 non-null  object
 4   item_id           500000 non-null  int64 
 5   item_name         500000 non-null  object
 6   playtime_forever  500000 non-null  int64 
 7   playtime_2weeks   500000 non-null  int64 
 8   genres_games      500000 non-null  object
dtypes: int64(5), object(4)
memory usage: 38.1+ MB


Unimos los dataFrame muestra_items y data_auxiliar por medio de la columna item_id

In [173]:
muestra_items=pd.merge(muestra_items, data_auxiliar, on='item_id', how='inner')

In [174]:
muestra_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,genres_games,year
0,kirbysmashed,896,76561198046016445,http://steamcommunity.com/id/kirbysmashed,282760,Circuits,0,0,Casual,2014
1,pimta,56,76561198071937362,http://steamcommunity.com/id/pimta,282760,Circuits,332,0,Casual,2014
2,CockInCider,1066,76561198058184559,http://steamcommunity.com/id/CockInCider,282760,Circuits,109,0,Casual,2014
3,LiamWA11,54,76561198081460083,http://steamcommunity.com/id/LiamWA11,282760,Circuits,58,0,Casual,2014
4,AntiLilac,244,76561198072772351,http://steamcommunity.com/id/AntiLilac,282760,Circuits,2,0,Casual,2014


## Función 1

In [175]:
def PlayTimeGenre( genero : str ):
    filto_genero=muestra_items[muestra_items['genres_games']==genero]  # hacemos una maskara con el genero_games
    anio_max_horas=filto_genero.groupby('year')['playtime_forever'].sum().idxmax() # hacemos un groupby y pedimos que sume por playtime_forever 
    
    resul= {'Año de lanzamiento con más horas jugadas para {}'.format(genero):anio_max_horas} # tranformamos el resultado en formato pedido
    return resul


In [176]:
PlayTimeGenre('Strategy')

{'Año de lanzamiento con más horas jugadas para Strategy': 2010}

## Funcion 2

In [152]:
def UserForGenre( genero : str ):
    filto_genero=muestra_items[muestra_items['genres_games']== genero]  # hacemos una maskara con el genero_games
    user_max_hora=filto_genero.groupby('user_id')['playtime_forever'].sum().idxmax() # hacemos groupby por user_id y por playtime_forever y que nos devuelva al usuario con mas horas jugadas
    
    # aplicamos maskarqa para cada año con el fin de determinar cuantas horas juego el usuario por año 
    anio_2010= muestra_items[(muestra_items['user_id'] ==user_max_hora) & (muestra_items['genres_games'] == genero) & (muestra_items['year']==2010)]['playtime_forever'].sum()
    anio_2011= muestra_items[(muestra_items['user_id'] ==user_max_hora) & (muestra_items['genres_games'] == genero) & (muestra_items['year']==2011)]['playtime_forever'].sum()
    anio_2012= muestra_items[(muestra_items['user_id'] ==user_max_hora) & (muestra_items['genres_games'] == genero) & (muestra_items['year']==2012)]['playtime_forever'].sum()
    anio_2013= muestra_items[(muestra_items['user_id'] ==user_max_hora) & (muestra_items['genres_games'] == genero) & (muestra_items['year']==2013)]['playtime_forever'].sum()
    anio_2014= muestra_items[(muestra_items['user_id'] ==user_max_hora) & (muestra_items['genres_games'] == genero) & (muestra_items['year']==2014)]['playtime_forever'].sum()
    anio_2015= muestra_items[(muestra_items['user_id'] ==user_max_hora ) & (muestra_items['genres_games'] == genero) & (muestra_items['year']==2015)]['playtime_forever'].sum()
    
    
    resultado = {
    'usuario': user_max_hora,
    'año 2010': anio_2010,
    'año 2011': anio_2011,
    'año 2012': anio_2012,
    'año 2013': anio_2013,
    'año 2014': anio_2014,
    'año 2015': anio_2015
    }
    return resultado

In [153]:
UserForGenre('Casual')

('usuario:',
 'tsunamitad',
 'año 2010:',
 0,
 'año 2011:',
 0,
 'año 2012:',
 0,
 'año 2013:',
 601647,
 'año 2014:',
 9,
 'año 2015:',
 0)

### Exportamos nuestra Data

In [178]:
muestra_items.to_csv('data_funcion_1y2.csv')

Importamos el dataFrame de Games

In [315]:
data_games=pd.read_csv('games.csv')

Renombramos la columna id por item_id del dataFrame data_games

In [316]:
data_games=data_games.rename(columns={'id':'item_id'})

Creamos una dataFrame auxiliar_2 que contenga las columnas de item_id publisher y title del dataFrame data_games

In [369]:
data_auxiliar_2=data_games[['item_id', 'developer','title']]

Cargamos nuestra data

In [370]:
data_review=pd.read_csv('reviews.csv')

Unimos el dataFrames data_review y data_auxiliar_2 por medio de la columna item_id

In [372]:
data_review=pd.merge(data_review, data_auxiliar_2, on='item_id', how='inner')

## Función 3

In [396]:
## variable año
def UsersRecommend( año : int ): 
    recomendado = data_review[(data_review['year']==año) & (data_review['recommend_True']==1) & ((data_review['sentiment_analysis']==1) | (data_review['sentiment_analysis']==2))]

    recomendado=recomendado.groupby(['item_id', 'title'])['sentiment_analysis'].sum().reset_index().sort_values(by='sentiment_analysis', ascending=False).head(3)

    result = recomendado['title']

    result= pd.DataFrame(result)

    puesto_1 = result.iloc[0]['title']
    puesto_2 = result.iloc[1]['title']
    puesto_3 = result.iloc[2]['title']

    resultado = {
        'PUESTO 1': puesto_1,
        'PUESTO 2': puesto_2,
        'PUESTO 3': puesto_3
    }

    return resultado

In [397]:
UsersRecommend(2015)

PUESTO 1: Counter-Strike: Global Offensive PUESTO 2: Team Fortress 2 PUESTO 3: Garry's Mod


## Función 4

In [402]:
def UsersWorstDeveloper( año : int ): 
    no_recomendado = data_review[(data_review['year'] == año) & (data_review['recommend_False'] == 1) & (data_review['sentiment_analysis'] == 0)]

    result= no_recomendado.groupby(['developer']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(3)

    puesto_1 = result.iloc[0]['developer']
    puesto_2 = result.iloc[1]['developer']
    puesto_3 = result.iloc[2]['developer']

    resultado = {
        'PUESTO 1': puesto_1,
        'PUESTO 2': puesto_2,
        'PUESTO 3': puesto_3
    }

    return resultado

In [403]:
UsersWorstDeveloper(2015)

PUESTO 1: Valve PUESTO 2: Bohemia Interactive PUESTO 3: Facepunch Studios


In [404]:
data_review.to_csv('data_funcion_3y4.csv')